# Creating Normalized Tables

## Imports

In [ ]:
import psycopg2

## Make Connection to DB

In [ ]:
try:
    conn = psycopg2.connect("dbname=####")
except psycopg2.Error as e:
    print('Error: Unable to establish a connection')
    print(e)
    
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print('Error: Cursor not connected')
    print(e)

conn.set_session(autocommit = True)

## Details of Table

* **Table Name**: music_library
* **Column 0**: Album ID
* **Column 1**: Album Name
* **Column 2**: Artist Name
* **Column 3**: Year
* **Column 4**: List of Songs

## Create Table (Not Normalized)

In [ ]:
query = 'CREATE TABLE IF NOT EXISTS music_library '
query += '(album_id int, album_name varchar, artist_name varchar, year int, songs text[]);'

try:
    cur.execute(query)
except psycopg2.Error as e:
    print('Error: Table not created')
    print(e)

In [ ]:
query = 'INSERT into music_library (album_id, album_name, artist_name, year, songs) '
query += 'VALUES (%s, %s, %s, %s, %s)'    

try:
    cur.execute(query, (1, 
                        'Rubber Soul', 
                        'The Beatles', 
                        1965, 
                        ['Michelle', 'Think for Yourself', 'In my Life']))
except psycopg2.Error as e:
    print('Error: Row not inserted')
    print(e)  

In [ ]:
try:
    cur.execute(query, (2, 
                        'Let It Be', 
                        'The Beatles', 
                        1970,
                        ['Let It Be', 'Across the Universe']))
except psycopg2.Error as e:
    print('Error: Row not inserted')
    print(e)

### Validate Insertion

In [ ]:
try:
    cur.execute('SELECT * FROM music_library;')
except psycopg2.Error as e:
    print('Error: SELECT *')
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

## Convert Table to `1NF`

In [ ]:
query = 'CREATE TABLE IF NOT EXISTS music_library_1nf '
query += '(album_id int, album_name varchar, artist_name varchar, year int, song_name varchar);'

try:
    cur.execute(query)
except psycopg2.Error as e:
    print('Error: Table not created')
    print(e)

In [ ]:
def insert_row(table, album_id, album_name, artist_name, year, song_name):
    query = f'INSERT into {table} (album_id, album_name, artist_name, year, song_name) '
    query += 'VALUES (%s, %s, %s, %s, %s)'    

    try:
        cur.execute(query, (album_id, 
                            album_name, 
                            artist_name, 
                            year, 
                            song_name))
    except psycopg2.Error as e:
        print('Error: Row not inserted')
        print(e)  

In [ ]:
insert_row('music_library_1nf',
           1,
           'Rubber Soul',
           'The Beatles',
           1965,
           'Michelle')

In [ ]:
insert_row('music_library_1nf',
           1,
           'Rubber Soul',
           'The Beatles',
           1965,
           'Think for Yourself')

In [ ]:
insert_row('music_library_1nf',
           1,
           'Rubber Soul',
           'The Beatles',
           1965,
           'In My Life')

In [ ]:
insert_row('music_library_1nf',
           2,
           'Let It Be',
           'The Beatles',
           1970,
           'Let It Be')

In [ ]:
insert_row('music_library_1nf',
           2,
           'Let It Be',
           'The Beatles',
           1970,
           'Across the Universe')

In [ ]:
query = "DELETE FROM music_library_1nf WHERE album_id = 2 AND song_name = 'Think for Yourself'"

cur.execute(query)

In [ ]:
try:
    cur.execute('SELECT DISTINCT * FROM music_library_1nf')
except psycopg2.Error as e:
    print(e)
    
row = cur.fetchone()

while row:
    print(row)
    row = cur.fetchone()

## Convert to `2NF`
* Break data into two tables: `album_library` and `song_library`

### Create `album_library` table

In [ ]:
query = 'CREATE TABLE IF NOT EXISTS album_library '
query += '(album_id int, album_name varchar, artist_name varchar, year int);'

try:
    cur.execute(query)
except psycopg2.Error as e:
    print(e)

In [ ]:
def insert_album(table, album_id, album_name, artist_name, year):
    query = f'INSERT into {table} (album_id, album_name, artist_name, year) '
    query += 'VALUES (%s, %s, %s, %s)'    

    try:
        cur.execute(query, (album_id, 
                            album_name, 
                            artist_name, 
                            year))
    except psycopg2.Error as e:
        print('Error: Row not inserted')
        print(e) 

In [ ]:
insert_album('album_library',
             1,
             'Rubber Soul',
             'The Beatles',
             1965)

In [ ]:
insert_album('album_library',
             2,
             'Let It Be',
             'The Beatles',
             1970)

In [ ]:
query = 'SELECT * FROM album_library'
cur.execute(query)

row = cur.fetchone()

while row:
    print(row)
    row = cur.fetchone()

### Create `song_library` table

In [ ]:
query = 'CREATE TABLE IF NOT EXISTS song_library '
query += '(song_id SERIAL, song_name varchar, album_id int)'

try:
    cur.execute(query)
except psycopg2.Error as e:
    print(e)

In [ ]:
def insert_song(table, song_name, album_id):
    query = f'INSERT into {table} (song_name, album_id) '
    query += 'VALUES (%s, %s)'    

    try:
        cur.execute(query, (song_name,
                            album_id))
    except psycopg2.Error as e:
        print('Error: Row not inserted')
        print(e) 
    

In [ ]:
insert_song('song_library', 'Michelle', 1)

In [ ]:
insert_song('song_library', 'Think for Yourself', 1)

In [ ]:
insert_song('song_library', 'In My Life', 1)

In [ ]:
insert_song('song_library', 'Let It Be', 2)

In [ ]:
insert_song('song_library', 'Across the Universe', 2)

In [ ]:
query = 'SELECT * FROM song_library'
try:
    cur.execute(query)
except psycopg2.Error as e:
    print(e)
    
row = cur.fetchone()

while row:
    print(row)
    row = cur.fetchone()

### Join Tables

In [ ]:
query = 'SELECT * FROM album_library a JOIN song_library s ON s.album_id = a.album_id'

try:
    cur.execute(query)
except psycopg2.Error as e:
    print(e)

In [ ]:
row = cur.fetchone()

while row:
    print(row)
    row = cur.fetchone()

## Convert to `3NF`
* Organize the data into three tables:
    1. album_library2
    2. song_library
    3. artist_library

### Create Tables

In [ ]:
query = 'CREATE TABLE IF NOT EXISTS album_library2 '
query += '(album_id SERIAL, album_name VARCHAR, artist_id INT, year INT)'

try:
    cur.execute(query)
except psycopg2.Error as e:
    print(e)

In [ ]:
query = 'CREATE TABLE IF NOT EXISTS song_library2 '
query += '(song_id SERIAL, song_name varchar, album_id int)'

try:
    cur.execute(query)
except psycopg2.Error as e:
    print(e)

In [ ]:
query = 'CREATE TABLE IF NOT EXISTS artist_library'
query += '(artist_id SERIAL, artist_name varchar)'

try:
    cur.execute(query)
except psycopg2.Error as e:
    print(e)

### Insert Data Into Tables

#### Artists

In [ ]:
query = "INSERT INTO artist_library (artist_name) VALUES ('The Beatles')"

try:
    cur.execute(query)
except psycopg2.Error as e:
    print(e)

#### Albums

In [ ]:
def insert_album(table, album_name, artist_id, year):
    query = f"INSERT INTO {table} (album_name, artist_id, year) VALUES (%s, %s, %s)"
    
    try:
        cur.execute(query, (album_name, artist_id, year))
    except psycopg2.Error as e:
        print(e)

In [ ]:
insert_album('album_library2', 'Rubber Soul', 1, 1965)

In [ ]:
insert_album('album_library2', 'Let It Be', 1, 1970)

#### Songs

In [ ]:
def insert_songs(table, album_id, song_name):
    query = f"INSERT INTO {table} (album_id, song_name) VALUES (%s, %s)"
    
    try:
        cur.execute(query, (album_id, song_name))
    except psycopg2.Error as e:
        print(e)

In [ ]:
insert_songs('song_library2',1, 'Michelle')

In [ ]:
insert_songs('song_library2', 1, 'Think for Yourself')

In [ ]:
insert_songs('song_library2', 1, 'In My Life')

In [ ]:
insert_songs('song_library2', 2, 'Let It Be')

In [ ]:
insert_songs('song_library2', 2, 'Across the Universe')

### Validate Queries

In [ ]:
def select(table):
    query = f'SELECT * FROM {table}'
    try:
        cur.execute(query)
    except psycopg2.Error as e:
        print(e)

    row = cur.fetchone()

    while row:
        print(row)
        row = cur.fetchone()

In [ ]:
select('album_library2')

In [ ]:
select('artist_library')

In [ ]:
select('song_library2')

## Close Connection

In [ ]:
cur.close()
conn.close()
